In [1]:
import argparse
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load nci


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for target_index in tqdm(np.arange(res.shape[dim])):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/60 [00:00<?, ?it/s]

epoch:   0 loss:0.702924 auc:0.4856
epoch:  20 loss:0.494176 auc:0.5104
epoch:  40 loss:0.404488 auc:0.5828
epoch:  60 loss:0.358112 auc:0.5828
epoch:  80 loss:0.332245 auc:0.5507
epoch: 100 loss:0.316429 auc:0.5771
epoch: 120 loss:0.303643 auc:0.5790
epoch: 140 loss:0.297601 auc:0.5946
epoch: 160 loss:0.291710 auc:0.5800
epoch: 180 loss:0.286714 auc:0.5515
epoch: 200 loss:0.284536 auc:0.5645
epoch: 220 loss:0.281118 auc:0.5598
epoch: 240 loss:0.277615 auc:0.5472
epoch: 260 loss:0.274232 auc:0.5692
epoch: 280 loss:0.277171 auc:0.5468
epoch: 300 loss:0.270490 auc:0.5641
epoch: 320 loss:0.268356 auc:0.5534
epoch: 340 loss:0.269411 auc:0.5462
epoch: 360 loss:0.268504 auc:0.5680
epoch: 380 loss:0.265549 auc:0.5273
epoch: 400 loss:0.261992 auc:0.5344
epoch: 420 loss:0.261335 auc:0.5251
epoch: 440 loss:0.258845 auc:0.5393
epoch: 460 loss:0.257846 auc:0.5319
epoch: 480 loss:0.257777 auc:0.4883
epoch: 500 loss:0.256361 auc:0.5156
epoch: 520 loss:0.257905 auc:0.4865
epoch: 540 loss:0.254992 auc

  2%|▏         | 1/60 [00:29<28:33, 29.03s/it]

Fit finished.
epoch:   0 loss:0.702065 auc:0.5346
epoch:  20 loss:0.494542 auc:0.6007
epoch:  40 loss:0.406006 auc:0.6116
epoch:  60 loss:0.357824 auc:0.5824
epoch:  80 loss:0.332256 auc:0.5863
epoch: 100 loss:0.321612 auc:0.5702
epoch: 120 loss:0.307360 auc:0.6007
epoch: 140 loss:0.296722 auc:0.6166
epoch: 160 loss:0.291787 auc:0.6115
epoch: 180 loss:0.286483 auc:0.6284
epoch: 200 loss:0.281385 auc:0.6380
epoch: 220 loss:0.278594 auc:0.6364
epoch: 240 loss:0.277260 auc:0.6312
epoch: 260 loss:0.276557 auc:0.6341
epoch: 280 loss:0.271583 auc:0.6417
epoch: 300 loss:0.270245 auc:0.6615
epoch: 320 loss:0.267662 auc:0.6402
epoch: 340 loss:0.267345 auc:0.6403
epoch: 360 loss:0.266338 auc:0.6271
epoch: 380 loss:0.263649 auc:0.6336
epoch: 400 loss:0.263488 auc:0.6625
epoch: 420 loss:0.259905 auc:0.6505
epoch: 440 loss:0.259842 auc:0.6664
epoch: 460 loss:0.259497 auc:0.6552
epoch: 480 loss:0.256126 auc:0.6559
epoch: 500 loss:0.257189 auc:0.6110
epoch: 520 loss:0.253669 auc:0.6637
epoch: 540 los

  3%|▎         | 2/60 [00:45<21:12, 21.93s/it]

...

Fit finished.
epoch:   0 loss:0.704589 auc:0.5401
epoch:  20 loss:0.498917 auc:0.5907
epoch:  40 loss:0.408076 auc:0.6687
epoch:  60 loss:0.361388 auc:0.6819
epoch:  80 loss:0.335659 auc:0.6908
epoch: 100 loss:0.318782 auc:0.6952
epoch: 120 loss:0.309014 auc:0.7040
epoch: 140 loss:0.299684 auc:0.7029
epoch: 160 loss:0.294790 auc:0.7135
epoch: 180 loss:0.290548 auc:0.7188
epoch: 200 loss:0.286863 auc:0.7210
epoch: 220 loss:0.282546 auc:0.7220
epoch: 240 loss:0.279008 auc:0.7167
epoch: 260 loss:0.276316 auc:0.7136
epoch: 280 loss:0.274536 auc:0.7179
epoch: 300 loss:0.273797 auc:0.7113
epoch: 320 loss:0.271081 auc:0.7078
epoch: 340 loss:0.270888 auc:0.7035
epoch: 360 loss:0.267874 auc:0.6987
epoch: 380 loss:0.266509 auc:0.6831
epoch: 400 loss:0.265551 auc:0.6819
epoch: 420 loss:0.267945 auc:0.6645
epoch: 440 loss:0.263840 auc:0.6570
epoch: 460 loss:0.260686 auc:0.6631
epoch: 480 loss:0.258753 auc:0.6577
epoch: 500 loss:0.257678 auc:0.6418
epoch: 520 loss:0.257518 auc:0.6303
epoch: 540 los

 17%|█▋        | 10/60 [02:59<14:02, 16.85s/it]

Fit finished.
epoch:   0 loss:0.700394 auc:0.5032
epoch:  20 loss:0.499352 auc:0.6960
epoch:  40 loss:0.408223 auc:0.7186
epoch:  60 loss:0.361670 auc:0.7304
epoch:  80 loss:0.333803 auc:0.7242
epoch: 100 loss:0.317151 auc:0.7059
epoch: 120 loss:0.306149 auc:0.7156
epoch: 140 loss:0.298074 auc:0.7102
epoch: 160 loss:0.292700 auc:0.6989
epoch: 180 loss:0.287890 auc:0.6970
epoch: 200 loss:0.284317 auc:0.7023
epoch: 220 loss:0.284402 auc:0.6917
epoch: 240 loss:0.278705 auc:0.6846
epoch: 260 loss:0.276768 auc:0.6863
epoch: 280 loss:0.273624 auc:0.6767
epoch: 300 loss:0.271319 auc:0.6855
epoch: 320 loss:0.269801 auc:0.6766
epoch: 340 loss:0.267881 auc:0.6829
epoch: 360 loss:0.267135 auc:0.6837
epoch: 380 loss:0.264844 auc:0.6567
epoch: 400 loss:0.264435 auc:0.6497
epoch: 420 loss:0.262800 auc:0.6428
epoch: 440 loss:0.260719 auc:0.6180
epoch: 460 loss:0.258740 auc:0.6189
epoch: 480 loss:0.256949 auc:0.5720
epoch: 500 loss:0.256448 auc:0.5712
epoch: 520 loss:0.254095 auc:0.5323
epoch: 540 los

 18%|█▊        | 11/60 [03:15<13:38, 16.70s/it]

Fit finished.
epoch:   0 loss:0.701745 auc:0.4929
epoch:  20 loss:0.494180 auc:0.7118
epoch:  40 loss:0.402332 auc:0.7517
epoch:  60 loss:0.357042 auc:0.7481
epoch:  80 loss:0.329646 auc:0.7443
epoch: 100 loss:0.313791 auc:0.7425
epoch: 120 loss:0.305465 auc:0.7393
epoch: 140 loss:0.297890 auc:0.7316
epoch: 160 loss:0.291705 auc:0.7292
epoch: 180 loss:0.287158 auc:0.7227
epoch: 200 loss:0.282831 auc:0.7258
epoch: 220 loss:0.279917 auc:0.7240
epoch: 240 loss:0.277273 auc:0.7210
epoch: 260 loss:0.275672 auc:0.7163
epoch: 280 loss:0.272447 auc:0.7121
epoch: 300 loss:0.271612 auc:0.7177
epoch: 320 loss:0.269217 auc:0.7035
epoch: 340 loss:0.267356 auc:0.7129
epoch: 360 loss:0.266260 auc:0.6978
epoch: 380 loss:0.264349 auc:0.7073
epoch: 400 loss:0.262951 auc:0.6873
epoch: 420 loss:0.260411 auc:0.6495
epoch: 440 loss:0.259553 auc:0.6089
epoch: 460 loss:0.259056 auc:0.5766
epoch: 480 loss:0.258399 auc:0.6012
epoch: 500 loss:0.256117 auc:0.5716
epoch: 520 loss:0.256456 auc:0.5016
epoch: 540 los

In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")